In [ ]:
from google.colab import files
upload = files.upload()

Saving spam.csv to spam.csv


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv('spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
X = df.v2
Y = df.v1
print([X.shape, Y.shape])

[(5572,), (5572,)]


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0,stratify = Y)

In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Embedding, LSTM, Dense, Dropout, Input
from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping

In [ ]:
max_words = 1000
max_len = 150
tk = Tokenizer(num_words=max_words)
tk.fit_on_texts(X_train)
sequences = tk.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
  inputs = Input(shape=[max_len])
  layer = Embedding(max_words, 50, input_length=max_len)(inputs)
  layer = LSTM(units=128)(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(512, activation='relu')(layer)
  layer = Dropout(0.5)(layer)
  outputs = Dense(1, activation='sigmoid')(layer)
  model = Model(inputs, outputs)
  return model

In [ ]:
model = RNN()
model.summary()
model.compile(optimizer=RMSprop(), loss = 'binary_crossentropy', metrics=['accuracy'])

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               91648     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               66048     
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 513 

In [ ]:
model.fit(sequences_matrix, Y_train, batch_size=128, epochs=20, validation_split=0.2, callbacks = [EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Train on 3565 samples, validate on 892 samples
Epoch 1/20
3565/3565 [==============================] - 17s 5ms/step - loss: 0.0465 - accuracy: 0.9879 - val_loss: 0.0357 - val_accuracy: 0.9922
Epoch 2/20
3565/3565 [==============================] - 17s 5ms/step - loss: 0.0332 - accuracy: 0.9902 - val_loss: 0.0390 - val_accuracy: 0.9910


In [ ]:
score = model.evaluate(sequences_matrix, Y_train)
print('Accuracy: %.3f [Loss: %.3f]' % (score[1], score[0]))

4457/4457 [==============================] - 7s 1ms/step
Accuracy: 0.993 [Loss: 0.027]


In [ ]:
test_sequences = tk.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
score = model.evaluate(test_sequences_matrix, Y_test)
print('Accuracy: %.3f [Loss: %.3f]' % (score[1], score[0]))

1115/1115 [==============================] - 2s 1ms/step
Accuracy: 0.985 [Loss: 0.071]


In [ ]:
testing_content = ["Oh k...i'm watching here:)"]

txt = tk.texts_to_sequences(testing_content)
txt_matrix = sequence.pad_sequences(txt, maxlen=max_len)
pred = model.predict(txt_matrix)
pred

array([[7.781852e-05]], dtype=float32)